In [1]:
!pip install pyxlsb --quiet
!pip install workalendar --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 75.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import pickle
import numpy as np
import pandas as pd

from datetime import datetime
from workalendar.america import Brazil

"""
Mouting Drive.
"""
drive.mount('/content/drive', force_remount=True)

df_forward_sales = pd.read_excel('./xlsb/base2023_venda.xlsb', engine="pyxlsb")

df_forward_sales = df_forward_sales.drop(columns=['Tipo Operação', 'Tipo', 'Valor Líquido'])

df_forward_sales = df_forward_sales.rename(columns={
    'Chave': 'id',
    'Cód. Título': 'ticker',
    'Cód. Cliente': 'client',
    'Cód. Corretora': 'broker',
    'Dt. Operação': 'execution_date',
    'Vencimento': 'maturity_date',
    'Quantidade': 'quantity',
    'Preço': 'price',
    'DI': 'di',
})


df_forward_sales['execution_date'] = pd.to_datetime(df_forward_sales['execution_date'], unit='D', origin='1899-12-30')
df_forward_sales['maturity_date'] = pd.to_datetime(df_forward_sales['maturity_date'], unit='D', origin='1899-12-30')

Mounted at /content/drive


In [3]:
brazil_calendar = Brazil()

def get_holidays(year):
    return [date for date, _ in brazil_calendar.holidays(year)]

def count_business_days(start_date, end_date, holidays):
    start_date = np.datetime64(start_date, 'D')
    end_date = np.datetime64(end_date, 'D')
    return np.busday_count(start_date, end_date, holidays=holidays)


years_in_dates = df_forward_sales['execution_date'].dt.year.unique().tolist()
holidays = []
for year in years_in_dates:
    holidays.extend(get_holidays(year))

holidays = np.array(holidays, dtype='datetime64[D]')

df_forward_sales['business_days_diff'] = df_forward_sales.apply(
    lambda row: count_business_days(row['execution_date'], row['maturity_date'], holidays),
    axis=1
)

In [4]:
def ideal_price(price_sale, business_days_diff, di_sale):
        return price_sale / ((1 + di_sale) ** (business_days_diff / 252))

df_forward_sales['ideal_price'] = ideal_price(df_forward_sales['price'], df_forward_sales['business_days_diff'], df_forward_sales['di'])

In [5]:
df_forward_sales['pending_quantity'] = df_forward_sales['quantity']

In [6]:
# I'm assuming you already have a DataFrame named 'df_forward_sales' with the columns mentioned.
# Make sure to convert 'execution_date' to string if it is not already.
exd = df_forward_sales['execution_date'].astype(str)

# Now, create the 'key' column by concatenating the required columns.
df_forward_sales['key'] = df_forward_sales['client'].astype(str) + df_forward_sales['ticker'].astype(str) + df_forward_sales['broker'].astype(str) + exd

# If you want to add a separator like an underscore or dash between the values, you can do it like this:
df_forward_sales['key'] = df_forward_sales['client'].astype(str) + '_' + df_forward_sales['ticker'].astype(str) + '_' + df_forward_sales['broker'].astype(str) + '_' + exd

df_forward_sales.insert(1, 'key', df_forward_sales.pop('key'))

In [7]:
df_forward_sales.head()

,id,key,client,execution_date,ticker,broker,quantity,price,maturity_date,di,business_days_diff,ideal_price,pending_quantity
0,1A,Cliente 4_LJQQ3_XPIN_2023-08-10,Cliente 4,2023-08-10,LJQQ3,XPIN,1073,6.049879,2023-09-11,0.131284,21,5.988008,1073
1,2A,Cliente 4_LJQQ3_XPIN_2023-08-10,Cliente 4,2023-08-10,LJQQ3,XPIN,1127,6.039911,2023-09-11,0.131284,21,5.978143,1127
2,3A,Cliente 4_LJQQ3_XPIN_2023-08-10,Cliente 4,2023-08-10,LJQQ3,XPIN,1330,6.039902,2023-09-11,0.131284,21,5.978134,1330
3,4A,Cliente 4_LJQQ3_XPIN_2023-08-10,Cliente 4,2023-08-10,LJQQ3,XPIN,1470,6.029939,2023-09-11,0.131284,21,5.968272,1470
4,5A,Cliente 4_LJQQ3_XPIN_2023-08-10,Cliente 4,2023-08-10,LJQQ3,XPIN,2375,6.039899,2023-09-11,0.131284,21,5.978130,2375


In [8]:
with open('./pickle/sales.pkl', 'wb') as handle:
    pickle.dump(df_forward_sales, handle, protocol=pickle.HIGHEST_PROTOCOL)

Purchases

In [9]:
# Read the Excel file
df_purchases_1 = pd.read_excel('./xlsb/base2023_compra.xlsb', engine="pyxlsb")
df_purchases_2 = pd.read_excel('./xlsb/base2023_compra_2.xlsb', engine="pyxlsb")

In [10]:
df_purchases_1['Dt. Operação'] = pd.to_datetime(df_purchases_1['Dt. Operação'], unit='D', origin='1899-12-30').dt.normalize()
df_purchases_2['Dt. Operação'] = pd.to_datetime(df_purchases_2['Dt. Operação'], format='%d/%m/%Y').dt.normalize()

df_purchases = pd.concat([df_purchases_1, df_purchases_2], ignore_index=True)

In [11]:
df_purchases = df_purchases.drop(columns=['Tipo Operação', 'Tipo', 'Valor Líquido', 'Dt. Liquidação', 'Vencimento'])

df_purchases = df_purchases.rename(columns={
    'Cód. Cliente': 'client',
    'Dt. Operação': 'execution_date',
    'Cód. Título': 'ticker',
    'Cód. Corretora': 'broker',
    'Quantidade': 'quantity',
    'Preço': 'price',
  })

In [12]:
# I'm assuming you already have a DataFrame named 'df_forward_sales' with the columns mentioned.
# Make sure to convert 'execution_date' to string if it is not already.
exdp = df_purchases['execution_date'].astype(str)

# Now, create the 'key' column by concatenating the required columns.
df_purchases['key'] = df_purchases['client'].astype(str) + df_purchases['ticker'].astype(str) + df_purchases['broker'].astype(str) + exdp

# If you want to add a separator like an underscore or dash between the values, you can do it like this:
df_purchases['key'] = df_purchases['client'].astype(str) + '_' + df_purchases['ticker'].astype(str) + '_' + df_purchases['broker'].astype(str) + '_' + exdp

In [13]:
df_purchases.insert(0, 'key', df_purchases.pop('key'))

In [14]:
df_purchases['matched'] = np.zeros(len(df_purchases))

In [15]:
df_purchases['available_quantity'] = df_purchases['quantity']

In [17]:
with open('./pickle/purchases.pkl', 'wb') as handle:
    pickle.dump(df_purchases, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
from tqdm import tqdm

with open('./pickle/sales.pkl', 'rb') as handle:
    df_sales = pickle.load(handle)

with open('./purchases.pkl', 'rb') as handle:
    df_purchases = pickle.load(handle)


df_sales['client'] = df_sales['client'].astype(str).str.strip()
df_sales['ticker'] = df_sales['ticker'].astype(str).str.strip()
df_sales['broker'] = df_sales['broker'].astype(str).str.strip()
df_sales['execution_date'] = pd.to_datetime(df_sales['execution_date'])

df_purchases['client'] = df_purchases['client'].astype(str).str.strip()
df_purchases['ticker'] = df_purchases['ticker'].astype(str).str.strip()
df_purchases['broker'] = df_purchases['broker'].astype(str).str.strip()
df_purchases['execution_date'] = pd.to_datetime(df_purchases['execution_date'])

pairs = {}

for index, row in tqdm(df_sales.iterrows(), total=(len(df_sales))):
    df_filtered = df_purchases[
        (df_purchases['key'] == row['key'])
    ]

    # Use the 'key' column to store the filtered DataFrame in the dictionary only if not empty
    if not df_filtered.empty:

        if row['key'] in pairs.keys():
            pairs[row['key']]['sale'].append(row)
        else:
            pairs[row['key']] = {
                'sale': [row],
                'purchase': df_filtered
            }

with open('./pairs.pkl', 'wb') as handle:
    pickle.dump(pairs, handle, protocol=pickle.HIGHEST_PROTOCOL)


100%|██████████| 44821/44821 [1:27:11<00:00,  8.57it/s]
